In [1]:
import sys

sys.path.insert(0, '..')
import os
from process_bigraph import pp, pf
from biosimulator_processes.data_model import *
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder


model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'

os.path.exists(model_filepath)

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
DeterministicTimeCourseParameterScan registered successfully.


True

In [2]:
class P:
    def __init__(self,
                 config=None):
        self.config = config
        if isinstance(self.config, dict):
            self.model = TimeCourseModel(**self.config['model'])

In [7]:
model = {
    'model': {
        'model_source': model_filepath
    }
}


tc = P(config=model)


In [9]:
pp(tc.model.model_dump()), type(tc.model.model_source)

{ 'model_changes': None,
  'model_id': '',
  'model_language': 'sbml',
  'model_name': 'Unnamed Composite Process TimeCourseModel',
  'model_source': { 'value': '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'},
  'model_units': '_default'}


(None, biosimulator_processes.data_model.ModelFilepath)

In [5]:
tc.model_id

''